In [122]:
from plasmapy import simulation

from plasmapy.formulary import magnetostatics

import astropy.units as u
import numpy as np
radius = 1 * u.m
main_current = 15 * u.MA
plasma_wire = magnetostatics.CircularWire([0, 0, 1], u.Quantity((0, 0, 0), u.m), radius, main_current)
plasma_wire

CircularWire(normal=[0. 0. 1.], center=[0. 0. 0.], radius=1.0, current=15000000.0)

In [123]:
n_coils = 4
coil_angles = np.linspace(0, 2*np.pi, n_coils, endpoint=False)
coil_angles

array([0.        , 1.57079633, 3.14159265, 4.71238898])

In [124]:
minor_radius = 0.3 * u.m
currents = u.Quantity(n_coils * [1e6], u.A)
currents

<Quantity [1000000., 1000000., 1000000., 1000000.] A>

In [125]:
coils = []
for i in range(n_coils):
    coil_angle = coil_angles[i]
    x = radius * np.cos(coil_angle)
    y = radius * np.sin(coil_angle)
    normal_angle = np.pi/2 + coil_angle
    normal = u.Quantity([np.cos(normal_angle), np.sin(normal_angle), 0])
    center = u.Quantity([x, y, 0 * u.m])
    coil = magnetostatics.CircularWire(normal, center, minor_radius, currents[i])
    coils.append(coil)

In [126]:
all_currents = coils + [plasma_wire]

# TODO Add functions to visualize our MagnetoStatics

Maybe I'm crazy but Mayavi could be a good fit!

# TODO units in reprs for MagnetoStatics

# TODO MagnetoStatics.magnetic_field accepts no units, just numpy arrays!

# TODO mendeleev vs atomic - comparison

In [127]:
plasma_wire.magnetic_field(center.value)

<Quantity [ 0.        ,  0.        , 10.45428585] T>

In [ ]:
from plasmapy.classes.plasma_base import GenericPlasma

E_unit = u.V / u.m
class Coils(GenericPlasma):
    """
    Work-in-progress class for passing analytical functions as fields 

    This is primarily helpful for `plasmapy.simulation.ParticleTracker`.
    """

    def __init__(self, *magnetostatics):
        """
        Initialize plasma paramters.
        The most basic description is composition (ion), temperature,
        density, and ionization.
        """
        self.magnetostatics = magnetostatics
    
    def interpolate_E(self, r: u.m):
        return u.Quantity(np.zeros(r.shape), E_unit)
    
    def interpolate_B(self, r: u.m):
        B = u.Quantity(np.zeros(r.shape), u.T)
        for ms in self.magnetostatics:
            field = ms.magnetic_field(r.si.value[0])
            B[0] += field
        return B
        

    @classmethod
    def is_datasource_for(cls, **kwargs):
        match = 'interpolate_B' in kwargs.keys()
        return match
    
c = Coils(*all_currents)

sim = simulation.ParticleTracker(c, 'e', dt=1e-8 * u.s, nt=int(1e5))
sim._x[0][0] = 1 + minor_radius.si.value / 2 # * (u.m / u.s)
sim._v[0][1] = 1000 # * (u.m / u.s)
sim._v[0][2] = 100 # * (u.m / u.s)


sim.run() # this should return a Solution object or sth

In [13]:
import mayavi
from mayavi import mlab
mlab.init_notebook()

In [111]:
fig = mlab.figure()
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig)
x, y, z = sim.position_history[:,0,:].T

trajectory = mlab.plot3d(x,y,z, sim.t, figure=fig, line_width=1e-4, representation='points')
mlab.colorbar(trajectory, title="Time")
mlab.orientation_axes(figure=fig)
display(fig)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [112]:
fig2 = mlab.figure()
x, y, z = sim.position_history[:,0,:].T

trajectory = mlab.plot3d(x,y,z, sim.t, figure=fig2, line_width=1e-13, representation='points')
mlab.colorbar(trajectory, title="Time")
mlab.orientation_axes(figure=fig2)
display(fig2)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [118]:
x = np.linspace(-1.2 * (radius + minor_radius), 1.2 * (radius + minor_radius), 50)
z = np.linspace(-1.2 * minor_radius, 1.2 * minor_radius, 50)
X, Y, Z = np.meshgrid(x, x, z, indexing='ij')

Bval = np.zeros((3, *Z.shape))

from tqdm import auto as tqdm

for i, xi in tqdm.tqdm(enumerate(x), total=len(x)):
    for j, yi in enumerate(x):
        for k, zi in enumerate(z):
            pos = u.Quantity([xi, yi, zi]).reshape(1, 3)
            field = c.interpolate_B(pos)
            Bval[:, i,j,k] = field.si.value

Bmag2 = np.sum(Bval**2, axis=0)

fig = mlab.figure()
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig)

contours = mlab.contour3d(X.value, Y.value, Z.value, Bmag2, figure=fig, contours = np.linspace(Bmag2.min(), Bmag2.max(), 50).tolist())
mlab.colorbar(contours, title="|B|^2")
mlab.orientation_axes(figure=fig)
display(fig)

Exception occurred in traits notification handler for object: <mayavi.tools.modules.IsoSurfaceFactory object at 0x7f9e974a95f0>, trait: contours, old value: 5, new value: [1.37437549e-15 6.63437070e-11 1.32686040e-10 1.99028372e-10
 2.65370705e-10 3.31713037e-10 3.98055370e-10 4.64397703e-10
 5.30740035e-10 5.97082368e-10 6.63424701e-10 7.29767033e-10
 7.96109366e-10 8.62451698e-10 9.28794031e-10 9.95136364e-10
 1.06147870e-09 1.12782103e-09 1.19416336e-09 1.26050569e-09
 1.32684803e-09 1.39319036e-09 1.45953269e-09 1.52587502e-09
 1.59221736e-09 1.65855969e-09 1.72490202e-09 1.79124435e-09
 1.85758669e-09 1.92392902e-09 1.99027135e-09 2.05661369e-09
 2.12295602e-09 2.18929835e-09 2.25564068e-09 2.32198302e-09
 2.38832535e-09 2.45466768e-09 2.52101001e-09 2.58735235e-09
 2.65369468e-09 2.72003701e-09 2.78637934e-09 2.85272168e-09
 2.91906401e-09 2.98540634e-09 3.05174867e-09 3.11809101e-09
 3.18443334e-09 3.25077567e-09]
Traceback (most recent call last):
  File "/home/dominik/minicond


Exception occurred in traits notification handler.
Please check the log file for details.


TraitError: The 'contours' trait of a Contour instance must be a list of items which are 1.3743754944193476e-15 <= a number <= 3.250775672527035e-09, but a value of array([1.37437549e-15, 6.63437070e-11, 1.32686040e-10, 1.99028372e-10,
       2.65370705e-10, 3.31713037e-10, 3.98055370e-10, 4.64397703e-10,
       5.30740035e-10, 5.97082368e-10, 6.63424701e-10, 7.29767033e-10,
       7.96109366e-10, 8.62451698e-10, 9.28794031e-10, 9.95136364e-10,
       1.06147870e-09, 1.12782103e-09, 1.19416336e-09, 1.26050569e-09,
       1.32684803e-09, 1.39319036e-09, 1.45953269e-09, 1.52587502e-09,
       1.59221736e-09, 1.65855969e-09, 1.72490202e-09, 1.79124435e-09,
       1.85758669e-09, 1.92392902e-09, 1.99027135e-09, 2.05661369e-09,
       2.12295602e-09, 2.18929835e-09, 2.25564068e-09, 2.32198302e-09,
       2.38832535e-09, 2.45466768e-09, 2.52101001e-09, 2.58735235e-09,
       2.65369468e-09, 2.72003701e-09, 2.78637934e-09, 2.85272168e-09,
       2.91906401e-09, 2.98540634e-09, 3.05174867e-09,

In [121]:
fig = mlab.figure()
for ms in all_currents:
    x, y, z = ms.curve(np.linspace(0, 2*np.pi))
    mlab.plot3d(x, y, z, figure=fig, line_width = 1e-3)

contours = mlab.contour3d(X.value, Y.value, Z.value, Bmag2, figure=fig, contours = np.linspace(Bmag2.min(), Bmag2.max(), 5).tolist())
mlab.colorbar(contours, title="|B|^2")
mlab.orientation_axes(figure=fig)
display(fig)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…